In [1]:
import pandas as pd
import numpy as np
import sys
from pathlib import Path
import json
from datetime import datetime

# Add shared modules to path
sys.path.append('../shared')

from simulation_utils import (
    SimulationConfig, 
    run_bfi_to_minimarker_simulation,
    retry_failed_participants,
    run_enhanced_bfi_to_minimarker_simulation
)
from schema_bfi2 import expanded_scale
from mini_marker_prompt import get_expanded_prompt

print("Setup complete!")
print(f"Analysis started at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")


Setup complete!
Analysis started at: 2025-07-13 23:04:21


In [2]:
# Load the statistically simulated BFI-2 data
data_path = Path('facet_lvl_simulated_data.csv')
if not data_path.exists():
    print(f"Simulated data file not found at {data_path}")
    print("Please run bfi2_facet_level_parameter_extraction_and_simulation.py first to generate the data")
    raise FileNotFoundError(f"Data file not found: {data_path}")

data = pd.read_csv(data_path)
print(f"Loaded simulated data shape: {data.shape}")
print(f"Columns: {list(data.columns[:10])}...")  # Show first 10 columns
print(f"BFI columns: {[col for col in data.columns if col.startswith('bfi') and col.replace('bfi', '').replace('_', '').isdigit()][:10]}...")  # Show BFI item columns
print(f"Value range: {data[[col for col in data.columns if col.startswith('bfi') and col.replace('bfi', '').replace('_', '').isdigit()]].min().min()} to {data[[col for col in data.columns if col.startswith('bfi') and col.replace('bfi', '').replace('_', '').isdigit()]].max().max()}")
data.head()


Loaded simulated data shape: (200, 80)
Columns: ['bfi1', 'bfi16', 'bfi31', 'bfi46', 'bfi6', 'bfi21', 'bfi36', 'bfi51', 'bfi11', 'bfi26']...
BFI columns: ['bfi1', 'bfi16', 'bfi31', 'bfi46', 'bfi6', 'bfi21', 'bfi36', 'bfi51', 'bfi11', 'bfi26']...
Value range: 1.0 to 5.0


,bfi1,bfi16,bfi31,bfi46,bfi6,bfi21,bfi36,bfi51,bfi11,bfi26,...,bfi_n_depression,bfi_n_emotional_volatility,bfi_o_intellectual_curiosity,bfi_o_aesthetic_sensitivity,bfi_o_creative_imagination,bfi_e,bfi_a,bfi_c,bfi_n,bfi_o
0,3.0,2.0,4.0,3.0,2.0,2.0,3.0,5.0,3.0,2.0,...,2.00,4.50,4.00,4.50,3.25,2.666667,3.833333,2.333333,3.750000,3.916667
1,4.0,2.0,3.0,3.0,3.0,3.0,3.0,2.0,3.0,4.0,...,2.75,4.50,2.50,3.25,3.00,3.250000,2.833333,4.083333,3.500000,2.916667
2,3.0,3.0,4.0,1.0,1.0,2.0,3.0,4.0,2.0,3.0,...,2.00,4.50,3.50,4.75,4.25,2.666667,2.750000,3.833333,3.833333,4.166667
3,5.0,1.0,2.0,5.0,5.0,5.0,1.0,1.0,2.0,2.0,...,4.25,2.25,4.75,4.25,3.75,4.583333,3.166667,2.916667,3.583333,4.250000
4,3.0,4.0,3.0,3.0,1.0,2.0,4.0,3.0,2.0,4.0,...,3.00,1.75,4.50,4.00,4.50,2.750000,4.333333,2.500000,3.000000,4.333333


In [3]:
# Generate BFI column list for processing
bfi_columns = [f"bfi{i}" for i in range(1, 61)]
print(f"Expected BFI columns: {len(bfi_columns)}")
print(f"Available BFI columns: {len([col for col in data.columns if col in bfi_columns])}")

# Verify all expected columns are present
missing_cols = [col for col in bfi_columns if col not in data.columns]
if missing_cols:
    print(f"Warning: Missing columns: {missing_cols}")
else:
    print("All BFI columns are present!")

# CRITICAL: For expanded format, we DO NOT apply reverse coding
# The simulated data is already in the correct format for expanded_scale mapping
print("\nNote: No reverse coding applied for expanded format - data already in correct format")


Expected BFI columns: 60
Available BFI columns: 60
All BFI columns are present!

Note: No reverse coding applied for expanded format - data already in correct format


In [4]:
# Map numeric values to Expanded format descriptions
def map_values_to_expanded(row):
    """Convert numeric BFI values to expanded format strings."""
    mapped_row = row.copy()
    for key in expanded_scale:
        if key in row.index and pd.notna(row[key]):  # Check if the key exists and value is not NaN
            index = int(row[key]) - 1  # Convert to 0-index (1-5 scale becomes 0-4 index)
            if 0 <= index < len(expanded_scale[key]):
                mapped_row[key] = expanded_scale[key][index]  # Replace with corresponding string
            else:
                print(f"Warning: Invalid index {index} for {key} with value {row[key]}")
    return mapped_row

print("Converting BFI values to Expanded format descriptions...")

# Apply the mapping function to BFI columns
mapped_data = data[bfi_columns].apply(map_values_to_expanded, axis=1)

# Combine all BFI descriptions into a single personality description
mapped_data['combined_bfi2'] = mapped_data[bfi_columns].apply(lambda row: ' '.join(row.astype(str)), axis=1)

# Add combined description to original data
data['combined_bfi2'] = mapped_data['combined_bfi2']

print("Expanded format personality descriptions created successfully")
print(f"Final data shape: {data.shape}")


Converting BFI values to Expanded format descriptions...
Expanded format personality descriptions created successfully
Final data shape: (200, 81)


/var/folders/zh/79y5ykz51rxcnjchzb73nfxw0000gn/T/ipykernel_89878/2585169807.py:9: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'You are very organized.' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  mapped_row[key] = expanded_scale[key][index]  # Replace with corresponding string
/var/folders/zh/79y5ykz51rxcnjchzb73nfxw0000gn/T/ipykernel_89878/2585169807.py:9: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'You are very disorganized.' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  mapped_row[key] = expanded_scale[key][index]  # Replace with corresponding string
/var/folders/zh/79y5ykz51rxcnjchzb73nfxw0000gn/T/ipykernel_89878/2585169807.py:9: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Y

In [12]:
# Preview a personality description
print("Sample Expanded format personality description:")
print("=" * 80)
sample_description = data.iloc[0]['combined_bfi2']
print(sample_description + "...")
print("=" * 80)
print(f"Full description length: {len(sample_description)} characters")


Sample Expanded format personality description:
You are somewhat outgoing, sociable. You are fairly compassionate usually soft-hearted. You are very organized. You are very relaxed handle stress very well. You have no artistic interests. You are not particularly assertive. You are fairly respectful usually treat others with respect. You are sometimes lazy. You become very pessimistic after experiencing a setback. You are curious about a large number of things. You sometimes feel excited or eager. You often find fault with others. You are somewhat dependable steady. You are very moody almost always have up and down mood swings. You are somewhat inventive sometimes find clever ways to do things. You are often quiet. You feel little sympathy for others. You are not particularly systematic rarely keep things in order. You are almost always tense. You are quite fascinated by art music or literature. You are fairly submissive often act as a follower. You sometimes start arguments with others

In [6]:
# Test prompt generation with first participant
first_participant = data.iloc[0]
sample_prompt = get_expanded_prompt(first_participant['combined_bfi2'])

print("=" * 80)
print("COMPLETE EXPANDED FORMAT PROMPT SENT TO LLM")
print("=" * 80)
print(sample_prompt)
print("=" * 80)
print(f"Prompt length: {len(sample_prompt)} characters")
print(f"Prompt word count: {len(sample_prompt.split())} words")


COMPLETE EXPANDED FORMAT PROMPT SENT TO LLM
### Your Assigned Personality ### 
Based on your detailed personality description below, please rate yourself on the following traits. Consider how each trait applies to you based on the personality profile provided.
You are somewhat outgoing, sociable. You are fairly compassionate usually soft-hearted. You are very organized. You are very relaxed handle stress very well. You have no artistic interests. You are not particularly assertive. You are fairly respectful usually treat others with respect. You are sometimes lazy. You become very pessimistic after experiencing a setback. You are curious about a large number of things. You sometimes feel excited or eager. You often find fault with others. You are somewhat dependable steady. You are very moody almost always have up and down mood swings. You are somewhat inventive sometimes find clever ways to do things. You are often quiet. You feel little sympathy for others. You are not particularly s

In [7]:
# Configuration for different models and temperatures
models_to_test = ['openai-gpt-3.5-turbo-0125',
                #   'gpt-4', 
                #   'gpt-4o', 
                #   'llama', 
                #   'deepseek'
				  ]
temperatures = [1]  # Use temperature 1 for stochastic responses
batch_size = 25  # Smaller batch size for stability across different APIs

# Create participant data list from DataFrame
participants_data = data.to_dict('records')

print(f"Simulation Configuration:")
print(f"  Models to test: {models_to_test}")
print(f"  Temperatures: {temperatures}")
print(f"  Batch size: {batch_size}")
print(f"  Total combinations: {len(models_to_test) * len(temperatures)}")
print(f"  Participants: {len(participants_data)}")

# Verify participant data structure
print(f"\nSample participant data keys: {list(participants_data[0].keys())[:10]}...")  # Show first 10 keys
print(f"Has 'combined_bfi2' key: {'combined_bfi2' in participants_data[0]}")


Simulation Configuration:
  Models to test: ['openai-gpt-3.5-turbo-0125']
  Temperatures: [1]
  Batch size: 25
  Total combinations: 1
  Participants: 200

Sample participant data keys: ['bfi1', 'bfi16', 'bfi31', 'bfi46', 'bfi6', 'bfi21', 'bfi36', 'bfi51', 'bfi11', 'bfi26']...
Has 'combined_bfi2' key: True


In [8]:
import threading
from concurrent.futures import ThreadPoolExecutor, as_completed
import time

# Thread-safe logging
log_lock = threading.Lock()

def safe_print(message, prefix="INFO"):
    """Thread-safe printing with timestamp and prefix"""
    timestamp = datetime.now().strftime("%H:%M:%S")
    with log_lock:
        print(f"[{timestamp}] {prefix}: {message}")

def run_simulation(model, temperature):
    """Run simulation for a single model-temperature combination."""
    simulation_id = f"{model}_temp{temperature}"
    
    # Start message
    safe_print(f"Starting simulation: {model} (temp={temperature})", "START")
    
    config = SimulationConfig(
        model=model,
        temperature=temperature,
        batch_size=batch_size,
        max_workers=10,
        max_retries=5,  # Enhanced retry logic
        base_wait_time=2.0,
        max_wait_time=60.0
    )
    
    start_time = time.time()
    
    try:
        # Use enhanced simulation with format validation and auto-retry
        results = run_enhanced_bfi_to_minimarker_simulation(
            participants_data=participants_data,
            config=config,
            output_dir="study_3_expanded_results",
            use_enhanced=True,  # Enable enhanced validation
            prompt_generator=get_expanded_prompt  # Use Expanded-specific prompts
        )
        
        # Check for failures
        failed_count = sum(1 for r in results if isinstance(r, dict) and 'error' in r)
        duration = time.time() - start_time
        
        if failed_count > 0:
            safe_print(f"Completed {simulation_id} in {duration:.1f}s - WARNING: {failed_count} participants failed", "WARN")
        else:
            safe_print(f"Completed {simulation_id} in {duration:.1f}s - All participants successful", "SUCCESS")
        
        return (simulation_id, results)
        
    except Exception as e:
        duration = time.time() - start_time
        safe_print(f"Failed {simulation_id} after {duration:.1f}s - Error: {str(e)}", "ERROR")
        return (simulation_id, {"error": str(e)})

# Main execution
print("=" * 80)
print("STARTING ENHANCED PARALLEL SIMULATIONS FOR STUDY 3 - EXPANDED FORMAT")
print(f"Models: {models_to_test}")
print(f"Temperatures: {temperatures}")
print(f"Total combinations: {len(models_to_test) * len(temperatures)}")
print(f"Participants: {len(participants_data)}")
print("=" * 80)

all_results = {}
start_time = time.time()

# Use ThreadPoolExecutor for parallel execution
with ThreadPoolExecutor(max_workers=len(models_to_test)) as executor:
    # Submit all jobs
    futures = [
        executor.submit(run_simulation, model, temperature)
        for model in models_to_test
        for temperature in temperatures
    ]
    
    # Collect results as they complete
    completed_count = 0
    total_jobs = len(futures)
    
    for future in as_completed(futures):
        key, result = future.result()
        all_results[key] = result
        completed_count += 1
        
        # Progress update
        safe_print(f"Progress: {completed_count}/{total_jobs} simulations completed", "PROGRESS")

total_duration = time.time() - start_time
print("\n" + "=" * 80)
print(f"ALL SIMULATIONS COMPLETED IN {total_duration:.1f} SECONDS")
print(f"Results keys: {list(all_results.keys())}")
print("=" * 80)


STARTING ENHANCED PARALLEL SIMULATIONS FOR STUDY 3 - EXPANDED FORMAT
Models: ['openai-gpt-3.5-turbo-0125']
Temperatures: [1]
Total combinations: 1
Participants: 200
[23:04:21] START: Starting simulation: openai-gpt-3.5-turbo-0125 (temp=1)
Starting simulation for 200 participants using openai-gpt-3.5-turbo-0125
Temperature: 1, Batch size: 25
Processing participants 0 to 24
Completed batch 0 to 24
Processing participants 25 to 49
Validation failed for participant 34 (attempt 1): Missing traits: ['Intellectual']
  Retrying in 1.0 seconds...
Completed batch 25 to 49
Processing participants 50 to 74
Completed batch 50 to 74
Processing participants 75 to 99
Completed batch 75 to 99
Processing participants 100 to 124
Completed batch 100 to 124
Processing participants 125 to 149
Completed batch 125 to 149
Processing participants 150 to 174
Completed batch 150 to 174
Processing participants 175 to 199
Completed batch 175 to 199
Results saved to study_3_expanded_results/bfi_to_minimarker_openai_

In [9]:
# Retry any failed participants
print("Checking for failed participants and retrying if necessary...")

retry_count = 0
for key, results in all_results.items():
    if isinstance(results, list):
        failed_count = sum(1 for r in results if isinstance(r, dict) and 'error' in r)
        if failed_count > 0:
            print(f"Retrying {failed_count} failed participants for {key}")
            retry_count += 1
            
            # Extract model and temperature from key
            model = key.split('_temp')[0]
            temperature = float(key.split('_temp')[1])
            
            config = SimulationConfig(
                model=model,
                temperature=temperature,
                batch_size=batch_size
            )
            
            updated_results = retry_failed_participants(
                results=results,
                participants_data=participants_data,
                prompt_generator=get_expanded_prompt,  # Use expanded-specific prompt function
                config=config,
                personality_key='combined_bfi2'
            )
            
            all_results[key] = updated_results
            
            # Save updated results
            from simulation_utils import save_simulation_results
            save_simulation_results(updated_results, "study_3_expanded_results", "bfi_to_minimarker", config)

if retry_count == 0:
    print("No failed participants found - all simulations successful!")
else:
    print(f"Retry process completed for {retry_count} model(s)")


Checking for failed participants and retrying if necessary...
No failed participants found - all simulations successful!


In [10]:
# Summary of all results
print("=" * 80)
print("STUDY 3 EXPANDED SIMULATION RESULTS SUMMARY")
print("=" * 80)

for key, results in all_results.items():
    if isinstance(results, list):
        total_participants = len(results)
        successful_participants = sum(1 for r in results if isinstance(r, dict) and 'error' not in r)
        failed_participants = total_participants - successful_participants
        
        print(f"\n{key}:")
        print(f"  Total participants: {total_participants}")
        print(f"  Successful: {successful_participants}")
        print(f"  Failed: {failed_participants}")
        print(f"  Success rate: {(successful_participants/total_participants)*100:.1f}%")
        
        # Sample a successful response for validation
        successful_responses = [r for r in results if isinstance(r, dict) and 'error' not in r]
        if successful_responses:
            sample_response = successful_responses[0]
            print(f"  Sample response keys: {list(sample_response.keys())[:10]}...")  # Show first 10 keys
            print(f"  Sample response length: {len(sample_response)} traits")
    else:
        print(f"\n{key}: FAILED - {results}")

print("\n" + "=" * 80)


STUDY 3 EXPANDED SIMULATION RESULTS SUMMARY

openai-gpt-3.5-turbo-0125_temp1:
  Total participants: 200
  Successful: 200
  Failed: 0
  Success rate: 100.0%
  Sample response keys: ['Bashful', 'Bold', 'Careless', 'Cold', 'Complex', 'Cooperative', 'Creative', 'Deep', 'Disorganized', 'Efficient']...
  Sample response length: 40 traits



In [11]:
# Save the preprocessed data for reference
output_path = Path('study_3_expanded_results')
output_path.mkdir(exist_ok=True)

# Save preprocessed data
data.to_csv(output_path / 'study3_expanded_preprocessed_data.csv', index=False)
print(f"Preprocessed data saved to {output_path / 'study3_expanded_preprocessed_data.csv'}")

# Save simulation metadata
metadata = {
    'simulation_type': 'study_3_expanded',
    'models_tested': models_to_test,
    'temperatures': temperatures,
    'batch_size': batch_size,
    'total_participants': len(participants_data),
    'simulation_date': datetime.now().isoformat(),
    'data_source': 'statistically_simulated_bfi2',
    'format': 'expanded',
    'key_differences_from_likert': [
        'No reverse coding applied - data already in correct format',
        'Uses expanded_scale mapping instead of likert_scale',
        'Uses get_expanded_prompt instead of get_likert_prompt',
        'Descriptions are narrative ("You are very...")',
    ],
    'results_summary': {
        key: {
            'total': len(results) if isinstance(results, list) else 0,
            'successful': sum(1 for r in results if isinstance(r, dict) and 'error' not in r) if isinstance(results, list) else 0
        } for key, results in all_results.items()
    }
}

with open(output_path / 'simulation_metadata.json', 'w') as f:
    json.dump(metadata, f, indent=2)

print(f"Simulation metadata saved to {output_path / 'simulation_metadata.json'}")

print("\n" + "=" * 60)
print("STUDY 3 EXPANDED FORMAT SIMULATION COMPLETE!")
print("\nNext steps:")
print("1. Run convergent validity analysis on the results")
print("2. Compare with Study 3 Likert results for format differences")
print("3. Compare with Study 2 results for study differences")
print("4. Results are saved in study_3_expanded_results/ directory")
print("5. Enhanced validation and retry logic handled format issues")
print("=" * 60)


Preprocessed data saved to study_3_expanded_results/study3_expanded_preprocessed_data.csv
Simulation metadata saved to study_3_expanded_results/simulation_metadata.json

STUDY 3 EXPANDED FORMAT SIMULATION COMPLETE!

Next steps:
1. Run convergent validity analysis on the results
2. Compare with Study 3 Likert results for format differences
3. Compare with Study 2 results for study differences
4. Results are saved in study_3_expanded_results/ directory
5. Enhanced validation and retry logic handled format issues
